This project is adapted from the assignment prompt provided in Coursera's course - NLP with Probabilistic Models.
It is based on the autocorrect model propsed by Peter Norvig in 2007

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import string

In [2]:
# function to extract words from a text file
def process_data(text_file):
    with open(text_file, 'r') as file:
        content = file.read()

    content=content.lower()
    vocab = re.findall('\w+',content)

    return vocab


In [3]:
word_l = process_data('shakespeare.txt')
vocab = set(word_l) 
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")


#process data works

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


In [6]:
# fn to create dictionary tracking frequency of words 
def get_count(word_l):
    vocab = set(word_l)
    ans=dict()
    for word in word_l:
        if word in ans.keys():
            ans[word]+=1
        else:
            ans[word]=1
    
    return ans

#fn to create dictionary with word probabilities
def get_probs(word_l):
    count = get_count(word_l)
    n = len(word_l)
    return {k: v/n for k, v in count.items()}


In [15]:
# edit functions - these functions return a list of possible words after replacing, inserting, deleting or switching letters
# only valid words that are a part of the text we have trained on are returned

def delete_letter(word, verbose=False):
    n=len(word)
    ans=[word[:i]+word[i+1:] for i in range(n)]
    splits = [(word[:i],word[i:]) for i in range(n)]

    if verbose: print(f"input word {word}, \nsplit_l = {splits}, \ndelete_l = {ans}")

    return ans

def switch_letter(word, verbose=False):

    n = len(word)
    ans=[word[:i-1]+word[i]+word[i-1]+word[i+1:] for i in range(1,n)]
    splits = [(word[:i],word[i:]) for i in range(n)]

    if verbose: print(f"input word {word}, \nsplit_l = {splits}, \ndelete_l = {ans}")

    return ans

def replace_letter(word,verbose=False):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    n=len(word)
    ans = [word[:i]+letter+word[i+1:] for i in range(n) for letter in letters if letter != word[i]]
    splits = [(word[:i],word[i:]) for i in range(n)]

    if verbose: print(f"input word {word}, \nsplit_l = {splits}, \ndelete_l = {ans}")

    return ans

def insert_letter(word,verbose=False):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    n=len(word)
    ans = [word[:i]+letter+word[i:] for i in range(n) for letter in letters]
    ans = ans + [word+letter for letter in letters]
    splits = [(word[:i],word[i:]) for i in range(n)]

    if verbose: print(f"input word {word}, \nsplit_l = {splits}, \ndelete_l = {ans}")

    return ans

In [27]:
# fns that return list of possible correct words after editing one letter or two letters at most
# words are returned in decreasing order of their probabilities

def edit_one_letter(word, vocab):
    ans = set()

    for word in replace_letter(word)+switch_letter(word)+insert_letter(word)+delete_letter(word):
        if word in vocab:
            ans.add(word)
    
    return ans

def edit_two_letter(word,vocab):
    
    first_edit= edit_one_letter(word,vocab)
    ans=set()
    for w in first_edit:
        ans = ans|edit_one_letter(w,vocab)
    
    return ans

def get_corrections(word, vocab,prob_dict,n=2, verbose = False):
    one_letter = edit_one_letter(word,vocab)
    two_letter = edit_two_letter(word,vocab)

    if word in vocab:
        # if word is correct, same returned
        return f'{word} is correct'
    #if candidates with one edit exist, candidates with 2 edits won't be considered, as one letter mistakes are more likely than 2 letter mistakes
    elif one_letter:
        suggestions = set(sorted(one_letter, key=lambda word: prob_dict.get(word, float('-inf')), reverse=True)[:n])
        k = len(suggestions)
        if k<n:
            #in case n one edit suggestions don't exist, the remaining ones will be given by 2 edits
            suggestions = suggestions | set(sorted(two_letter, key=lambda word: prob_dict.get(word, float('-inf')), reverse=True)[:n-k])
        return suggestions
    elif two_letter:
        return set(sorted(two_letter, key=lambda word: prob_dict.get(word, float('-inf')), reverse=True)[:n])
    else:

        return f'Sorry, no words like {word} exist in the given training set'

Finally, test the autocorrect function yourself!!

In [28]:
#Test implementation
your_word = re.sub(r'[^a-z]', '', input('Enter word: ').lower())
n = int(input('Number of suggestions desired: ')) 
prob_dict=get_probs(word_l)
tmp_corrections = get_corrections(your_word, vocab, prob_dict, n, verbose=True) # keep verbose=True
print('Autocorrect suggestions: ', tmp_corrections)

Autocorrect suggestions:  never is correct


Lastly, the following function can be used to find the min edit distance between two words using dynamic programming.

In [12]:
def min_edit_distance(source,target):

    m=len(source)+1
    n = len(target)+1
    dp_table = [[[0] for i in range(n)] for j in range(m)]

    for i in range(m):
        dp_table[i][0]=i
    for i in range(1,n):
        dp_table[0][i]=i
    
    for i in range(1,m):
        for j in range(1,n):
            switched=0
            #cost to replace letters
            if source[i-1] == target[j-1]:
                replace_cost=0
            else:
                replace_cost=1
            
            if i<m-1 and j<n-1:
                #cost to switch letters
                if (source[i-1] , source[i] == target[j], target[j-1]):
                    switch_cost = 1
                    switched = 1
                else:
                    switch_cost = float(inf)
                
                if switched ==0:
                    dp_table[i][j]=min([dp_table[i-1][j]+1, dp_table[i-1][j]+1, dp_table[i-1][j-1]+replace_cost,dp_table[i-1][j-1]+switch_cost])
                else:
                    dp_table[i][j]=min([dp_table[i-1][j]+1, dp_table[i-1][j]+1, dp_table[i-1][j-1]+replace_cost,dp_table[i-1][j-1]])
                    switched=0
            else:
                dp_table[i][j]=min([dp_table[i-1][j]+1, dp_table[i-1][j]+1, dp_table[i-1][j-1]+replace_cost])

    
    return dp_table[-1][-1], dp_table

In [14]:
min_edit_distance('othger','other')

(1,
 [[0, 1, 2, 3, 4, 5],
  [1, 0, 1, 2, 3, 5],
  [2, 1, 0, 1, 2, 4],
  [3, 2, 1, 0, 1, 3],
  [4, 3, 2, 1, 0, 2],
  [5, 4, 3, 2, 1, 1],
  [6, 5, 4, 3, 2, 1]])